# Finetune Bert on QuAC
**Student Credentials:** sdi1800119, Vissarion Moutafis


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn
import re
seaborn.set_style("ticks")

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report

import torch
import torch.nn as nn
import torchtext
from torch.utils.data import SubsetRandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

!pip install transformers datasets
!apt install git-lfs
import transformers
import datasets
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification, AutoModelForQuestionAnswering
from datasets import load_dataset, load_metric

!pip install tqdm
from tqdm import tqdm, trange

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

     |████████████████████████████████| 311 kB 924 kB/s            
     |████████████████████████████████| 212 kB 14.8 MB/s            



The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 2s (2139 kB/s)

78Selecting previously unselected package git-lfs.
(Reading database ... 103272 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking git-lfs (2.9.2-1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up git-lfs (2.9.2-1) ...
7Progress: [ 60%] [

## Load the model and the dataset

In [ ]:
model_checkpoint = 'bert-base-uncased'

In [ ]:
train_dataset = load_dataset('../input/squadlikeloader/squad_like.py', data_files={'train':'../quactosquad/quac_train.json', 'validation':'../quactosquad/quac_val.json'}, split='train')

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_like downloaded and prepared to /root/.cache/huggingface/datasets/squad_like/default-94886529e6fd8abb/0.0.0/c11bde73ef00f53b085b6a086d13514938f65b80af061fc874ce3e7514c24892. Subsequent calls will reuse this data.


In [ ]:
test_dataset = load_dataset('squad_v2', split='validation')

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


## Preprocess the dataset

In [ ]:
from transformers import AutoTokenizer
# initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
MAX_LENGTH = 384
DOC_STRIDE = 128 # multi-context overlapping range for large context'd instances 


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def preprocess_squad(examples):
  # get the questions and the context
  questions = [q.strip() for q in examples["question"]]
  context = examples["context"]
  # tokenize questions along with the context 
  inputs = tokenizer(
        questions,
        context,
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_overflowing_tokens=True
    )
  offset_mapping = inputs.pop("offset_mapping")
  sample_mapping = inputs.pop("overflow_to_sample_mapping")
  answers = examples["answers"]
  start_positions = []
  end_positions = []
  
  for i, offset in enumerate(offset_mapping):
    sample_index = sample_mapping[i]
    answer = examples["answers"][sample_index]
    # if there is no answer default to [CLS]
    if not answer["answer_start"]:
      start_positions.append(inputs['input_ids'][i].index(tokenizer.cls_token_id))
      end_positions.append(inputs['input_ids'][i].index(tokenizer.cls_token_id))
      continue 
    
    # get answer start and end positions
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
      idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label it (0, 0)
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
#       start_positions.append(inputs['input_ids'][i].index(tokenizer.cls_token_id))
#       end_positions.append(inputs['input_ids'][i].index(tokenizer.cls_token_id))
      start_positions.append(-1)
      end_positions.append(-1)
    else:
      # Otherwise it's the start and end token positions
      idx = context_start
      while idx <= context_end and offset[idx][0] <= start_char:
        idx += 1
      start_positions.append(idx - 1)

      idx = context_end
      while idx >= context_start and offset[idx][1] >= end_char:
        idx -= 1
      end_positions.append(idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  return inputs

In [ ]:
train_dataset = train_dataset.map(preprocess_squad, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_squad, batched=True, remove_columns=test_dataset.column_names)

  0%|          | 0/84 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
def is_valid(s, e):
    return s != -1 and e != -1

train_dataset = train_dataset.filter(is_valid, input_columns=['start_positions', 'end_positions'])
test_dataset = test_dataset.filter(is_valid, input_columns=['start_positions', 'end_positions'])


  0%|          | 0/84 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

## Fine Tuning

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=24, shuffle=True)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
torch.cuda.empty_cache()
model.to(device)
model.train()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
def training_step(model, optimizer, epoch_i, train_loader, history=None):
  torch.cuda.empty_cache()
  # 1 step of backprop with train/test error estimation
  total_loss = 0
  acc = []
  pbar = tqdm(train_loader)
  for i,batch in enumerate(pbar):
    torch.cuda.empty_cache()
    # set the gradients to zero for new estimation
    optimizer.zero_grad() 
    # forward pass
    args = {
        "start_positions" : torch.LongTensor(batch["start_positions"]).to(device),
        "end_positions" : torch.LongTensor(batch["end_positions"]).to(device),
        "input_ids" : torch.stack(batch["input_ids"], axis=1).to(device),
        "attention_mask" : torch.stack(batch["attention_mask"], axis=1).to(device),
    }
    outputs = model(**args) 
    loss = outputs[0]
    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)
    # compute loss 
    total_loss += loss.item()

    # backpropagate error
    loss.backward()
    # apply gradient clipping adjust model's parameters
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
    optimizer.step()
    
    acc.append(((start_pred == args['start_positions']).sum()/len(start_pred)).item())
    acc.append(((end_pred == args['end_positions']).sum()/len(end_pred)).item())
    pbar.set_description('Epoch {}: train loss: {}, train accuracy: {}%'.format(epoch_i, total_loss/(i+1), sum(acc)/len(acc)), refresh=True)
    
  train_loss = total_loss/len(train_loader)
    
    # if the user provides with a history dict the training step will save the current epoch's train-test loss
  if history is not None: 
    history['train'].append(train_loss)
  
  test_loss = 0
  acc = []
  pbar = tqdm(test_loader)
  for i,batch in enumerate(pbar):
    torch.cuda.empty_cache()
    with torch.no_grad():
      args = {
          "start_positions" : torch.LongTensor(batch["start_positions"]).to(device),
          "end_positions" : torch.LongTensor(batch["end_positions"]).to(device),
          "input_ids" : torch.stack(batch["input_ids"], axis=1).to(device),
          "attention_mask" : torch.stack(batch["attention_mask"], axis=1).to(device)
        }
      outputs = model(**args)
      test_loss += outputs[0].item()
      start_pred = torch.argmax(outputs['start_logits'], dim=1)
      end_pred = torch.argmax(outputs['end_logits'], dim=1)
      acc.append(((start_pred == args['start_positions']).sum()/len(start_pred)).item())
      acc.append(((end_pred == args['end_positions']).sum()/len(end_pred)).item())
      pbar.set_description('Epoch {}: test loss {}%, test accuracy {}%'.format(epoch_i, test_loss/(i+1), sum(acc)/len(acc)), refresh=True)
        
  if history is not None: 
    history['test'].append(test_loss/len(test_loader))
    

  
  return train_loss, test_loss/len(test_loader), sum(acc)/len(acc)

def train_model(history, model, train_loader, epochs, _lr):
  # set to training mode
  model.train()
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.AdamW(model.parameters(), lr = _lr)

  for epoch in range(epochs):
    training_step(model, optimizer, epoch, train_loader, history)

In [ ]:
history = {'train':[], 'test':[]}
train_model(history, model, train_loader, 2,  3e-5)

Epoch 0: train loss: 3.3921460354744397, train accuracy: 0.3606420633146933%: 100%|██████████| 1747/1747 [39:19<00:00,  1.35s/it]
Epoch 0: test loss 3.4228402898606074%, test accuracy 0.4569223232240791%: 100%|██████████| 502/502 [03:04<00:00,  2.73it/s]
Epoch 1: train loss: 3.1554847793983063, train accuracy: 0.37926923389678247%: 100%|██████████| 1747/1747 [39:32<00:00,  1.36s/it]
Epoch 1: test loss 3.7013701684921387%, test accuracy 0.36824923567102846%: 100%|██████████| 502/502 [03:05<00:00,  2.71it/s]


## Upload the model

In [ ]:
model.push_to_hub('vissa/bert-finetuned-quac', use_auth_token='YOUR_WRT_TOKEN_HERE')

/opt/conda/lib/python3.7/site-packages/huggingface_hub/hf_api.py:726: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/vissa/bert-finetuned-quac into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/415M [00:00<?, ?B/s]

To https://huggingface.co/vissa/bert-finetuned-quac
   b57f495..b4c1a7d  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/vissa/bert-finetuned-quac/commit/b4c1a7dc870bdbdfe73a95c5c30c2af907fd7f22'

In [ ]:
model.save_pretrained('./' , private=True)